In [1]:
import torch
import torch_pruning as tp
from yolox.models import create_yolox_model

model = create_yolox_model(num_classes=20,name = 'yolox_custom',  
exp_path = '/workspace/tensorrt/YOLOX/exps/default/custom2/type1/all25_stem/yolox_voc_tiny_all25_stem.py',
ckpt_path = '/workspace/tensorrt/YOLOX/fpgm_2/type1/first/yolox_voc_tiny_all25_stem/best_ckpt.pth'
).cuda()
# 1. setup strategy (L1 Norm)
strategy = tp.strategy.L1Strategy() # or tp.strategy.RandomStrategy()

/usr/local/lib/python3.8/dist-packages/tqdm-4.64.0-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# 2. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224).cuda())



In [3]:

temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2)
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('0.0','0[0]')
            temp2[i] = temp2[i].replace('0.1','0[1]')
            temp2[i] = temp2[i].replace('1.0','1[0]')
            temp2[i] = temp2[i].replace('1.1','1[1]')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝


['backbone.backbone.stem.conv.conv', 'backbone.backbone.dark2[0].conv', 'backbone.backbone.dark2[1].conv1.conv', 'backbone.backbone.dark2[1].conv2.conv', 'backbone.backbone.dark2[1].conv3.conv', 'backbone.backbone.dark2[1].m[0].conv1.conv', 'backbone.backbone.dark2[1].m[0].conv2.conv', 'backbone.backbone.dark3[0].conv', 'backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[0].conv2.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv2.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv2.conv', 'backbone.backbone.dark4[0].conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[0].conv2.conv', 'backbone.backbone.dark4[1].m

In [4]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [5]:
from pruning_imagenet import Mask

In [6]:
m = Mask(model)
m.mask_index = [24,27,30,33,39,45,54,57,60,63,69,75,90,93,99,102,96  # backbone conv3 30 60 96
                            ,105,108,111,114,117,120,123,141,159] # neck

m.mask_index = [24,27,30,33,39,45,54,57,60,63,69,75,90,93,99,102,96 
                            ,105,108,111,114,117,120,123,141,159,126,129,132,135,138,144,147,150,153,156,162,165,168,171,174, #neck
                            231,234,237, #stems
                            177,180,183,186,189,192,195,198,201,204,207,210] # head
        

In [7]:
m.init_length()

In [8]:
m.init_mask(1, 1/3)
print(m.mask_index)

filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done


In [9]:
m.do_mask()
m.do_similar_mask()

mask Done
mask similar Done


In [10]:
m.if_zero()
print(m.mask_index)
print(m.similar_matrix)

layer: 24, number of nonzero weight is 2304, zero is 1152
layer: 27, number of nonzero weight is 2304, zero is 1152
layer: 30, number of nonzero weight is 3456, zero is 1728
layer: 33, number of nonzero weight is 864, zero is 432
layer: 39, number of nonzero weight is 864, zero is 432
layer: 45, number of nonzero weight is 864, zero is 432
layer: 54, number of nonzero weight is 9216, zero is 4608
layer: 57, number of nonzero weight is 9216, zero is 4608
layer: 60, number of nonzero weight is 13824, zero is 6912
layer: 63, number of nonzero weight is 3456, zero is 1728
layer: 69, number of nonzero weight is 3456, zero is 1728
layer: 75, number of nonzero weight is 3456, zero is 1728
layer: 90, number of nonzero weight is 36864, zero is 18432
layer: 93, number of nonzero weight is 36864, zero is 18432
layer: 96, number of nonzero weight is 55296, zero is 27648
layer: 99, number of nonzero weight is 13824, zero is 6912
layer: 102, number of nonzero weight is 124416, zero is 62208
layer: 1

In [11]:
model = m.model


In [12]:
dict ={}


for key, value in model.state_dict().items():
    temp = []
    if len(value.shape) ==4:
        print('pruning channel and layer name : ',len(value),',', key)
        for i in range(len(value)):
            if value[i].sum() == 0: 
                print('pruning channex index : ', i)
                #temp = torch.norm(value.view(len(value), -1) , p= 1, dim=1)
                #print(temp)
                #dict[key].append(i)
                temp.append(i)
    if temp:   
        dict[key]  = temp



pruning channel and layer name :  24 , backbone.backbone.stem.conv.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.0.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv2.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.1.conv3.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv2.conv.weight
pruning channel and layer name :  96 , backbone.backbone.dark3.0.conv.weight
pruning channel and layer name :  36 , backbone.backbone.dark3.1.conv1.conv.weight
pruning channex index :  1
pruning channex index :  3
pruning channex index :  5
pruning channex index :  12
pruning channex index :  13
pruning channex index :  14
pruning channex index :  23
pruning channex index :  25
pruning channex index :  26
pruning chan

In [13]:
print(dict)
#del dict['backbone.backbone.dark3.1.conv3.conv.weight'][0]

{'backbone.backbone.dark3.1.conv1.conv.weight': [1, 3, 5, 12, 13, 14, 23, 25, 26, 31, 32, 33], 'backbone.backbone.dark3.1.conv2.conv.weight': [0, 1, 2, 3, 4, 6, 8, 14, 15, 16, 28, 34], 'backbone.backbone.dark3.1.conv3.conv.weight': [2, 3, 6, 11, 13, 16, 22, 25, 33, 34, 36, 38, 39, 41, 43, 45, 51, 52, 54, 60, 63, 68, 70, 71], 'backbone.backbone.dark3.1.m.0.conv1.conv.weight': [3, 6, 7, 13, 17, 18, 20, 25, 27, 28, 32, 33], 'backbone.backbone.dark3.1.m.1.conv1.conv.weight': [2, 3, 8, 9, 11, 14, 17, 18, 23, 26, 30, 33], 'backbone.backbone.dark3.1.m.2.conv1.conv.weight': [1, 4, 10, 12, 14, 16, 18, 22, 24, 28, 30, 34], 'backbone.backbone.dark4.1.conv1.conv.weight': [4, 7, 11, 12, 13, 17, 19, 20, 21, 23, 24, 28, 32, 33, 34, 40, 42, 48, 52, 53, 54, 57, 64, 67], 'backbone.backbone.dark4.1.conv2.conv.weight': [0, 3, 6, 11, 13, 14, 15, 18, 24, 26, 32, 33, 34, 42, 43, 47, 49, 51, 55, 56, 58, 63, 68, 69], 'backbone.backbone.dark4.1.conv3.conv.weight': [1, 3, 5, 11, 12, 13, 16, 17, 19, 21, 23, 24, 2

In [14]:

temp = []
temp2 = list(dict.keys())
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('.0.0','[0][0]')
            temp2[i] = temp2[i].replace('.0.1','[0][1]')
            temp2[i] = temp2[i].replace('.1.0','[1][0]')
            temp2[i] = temp2[i].replace('.1.1','[1][1]')
            temp2[i] = temp2[i].replace('.2.0','[2][0]')
            temp2[i] = temp2[i].replace('.2.1','[2][1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp2[i] = temp2[i].replace('.0','[0]')
            temp2[i] = temp2[i].replace('.1','[1]')
            temp2[i] = temp2[i].replace('.2','[2]')
            temp.append(temp2[i])

#temp.insert(27,'head.stems[0].conv')
#temp.insert(28,'head.stems[1].conv')
#temp.insert(29,'head.stems[2].conv')

del temp[-1]
del temp[-1]
del temp[-1]

temp.remove('head.cls_convs[2][1].conv')
temp.remove('head.reg_convs[2][1].conv')
temp.remove('head.reg_convs[1][1].conv')
temp.remove('head.cls_convs[1][1].conv')
temp.remove('head.reg_convs[0][1].conv')
temp.remove('head.cls_convs[0][1].conv')
#temp.remove('head.cls_convs[1][0].conv')
#temp.remove('head.reg_convs[1][0].conv')
import copy
key = []
temp2 = copy.deepcopy(temp)
for i in range(len(temp2)):
      
            temp2[i] = temp2[i].replace('[0][0]','.0.0')
            temp2[i] = temp2[i].replace('[0][1]','.0.1',)
            temp2[i] = temp2[i].replace('[1][0]','.1.0',)
            temp2[i] = temp2[i].replace('[1][1]','.1.1',)
            temp2[i] = temp2[i].replace('[2][0]','.2.0',)
            temp2[i] = temp2[i].replace('[2][1]','.2.1',)
            temp2[i] = temp2[i].replace('3[0]','3.0',)
            temp2[i] = temp2[i].replace('3[1]','3.1')
            temp2[i] = temp2[i].replace('4[0]','4.0',)
            temp2[i] = temp2[i].replace('4[1]','4.1',)
            temp2[i] = temp2[i].replace('5[0]','5.0',)
            temp2[i] = temp2[i].replace('5[1]','5.1',)
            temp2[i] = temp2[i].replace('5[2]','5.2',)
            temp2[i] = temp2[i].replace('m[0]','m.0',)
            temp2[i] = temp2[i].replace('m[1]','m.1',)
            temp2[i] = temp2[i].replace('m[2]','m.2',)
            temp2[i] = temp2[i].replace('[0]','.0',)
            temp2[i] = temp2[i].replace('[1]','.1',)
            temp2[i] = temp2[i].replace('[2]','.2',)
            key.append(temp2[i])

print(key)
print(temp)

['backbone.backbone.dark3.1.conv1.conv', 'backbone.backbone.dark3.1.conv2.conv', 'backbone.backbone.dark3.1.conv3.conv', 'backbone.backbone.dark3.1.m.0.conv1.conv', 'backbone.backbone.dark3.1.m.1.conv1.conv', 'backbone.backbone.dark3.1.m.2.conv1.conv', 'backbone.backbone.dark4.1.conv1.conv', 'backbone.backbone.dark4.1.conv2.conv', 'backbone.backbone.dark4.1.conv3.conv', 'backbone.backbone.dark4.1.m.0.conv1.conv', 'backbone.backbone.dark4.1.m.1.conv1.conv', 'backbone.backbone.dark4.1.m.2.conv1.conv', 'backbone.backbone.dark5.2.conv1.conv', 'backbone.backbone.dark5.2.conv2.conv', 'backbone.backbone.dark5.2.conv3.conv', 'backbone.backbone.dark5.2.m.0.conv1.conv', 'backbone.backbone.dark5.2.m.0.conv2.conv', 'backbone.lateral_conv0.conv', 'backbone.C3_p4.conv1.conv', 'backbone.C3_p4.conv2.conv', 'backbone.C3_p4.conv3.conv', 'backbone.C3_p4.m.0.conv1.conv', 'backbone.C3_p4.m.0.conv2.conv', 'backbone.reduce_conv1.conv', 'backbone.C3_p3.conv1.conv', 'backbone.C3_p3.conv2.conv', 'backbone.C3_p3

In [15]:
value = list(dict.values())
value

[[1, 3, 5, 12, 13, 14, 23, 25, 26, 31, 32, 33],
 [0, 1, 2, 3, 4, 6, 8, 14, 15, 16, 28, 34],
 [2,
  3,
  6,
  11,
  13,
  16,
  22,
  25,
  33,
  34,
  36,
  38,
  39,
  41,
  43,
  45,
  51,
  52,
  54,
  60,
  63,
  68,
  70,
  71],
 [3, 6, 7, 13, 17, 18, 20, 25, 27, 28, 32, 33],
 [2, 3, 8, 9, 11, 14, 17, 18, 23, 26, 30, 33],
 [1, 4, 10, 12, 14, 16, 18, 22, 24, 28, 30, 34],
 [4,
  7,
  11,
  12,
  13,
  17,
  19,
  20,
  21,
  23,
  24,
  28,
  32,
  33,
  34,
  40,
  42,
  48,
  52,
  53,
  54,
  57,
  64,
  67],
 [0,
  3,
  6,
  11,
  13,
  14,
  15,
  18,
  24,
  26,
  32,
  33,
  34,
  42,
  43,
  47,
  49,
  51,
  55,
  56,
  58,
  63,
  68,
  69],
 [1,
  3,
  5,
  11,
  12,
  13,
  16,
  17,
  19,
  21,
  23,
  24,
  25,
  30,
  34,
  35,
  38,
  40,
  43,
  46,
  48,
  55,
  56,
  58,
  63,
  69,
  70,
  71,
  75,
  76,
  88,
  90,
  95,
  97,
  98,
  103,
  106,
  109,
  110,
  111,
  113,
  114,
  119,
  130,
  131,
  135,
  137,
  142],
 [2,
  6,
  8,
  12,
  16,
  19,
  20,

In [16]:
for i in range(len(temp)):
    pruning_plan = DG.get_pruning_plan(eval(f"model.{temp[i]}"), tp.prune_conv_out_channel, dict[(key[i]) + '.weight'])
    print(pruning_plan)
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()     

Conv2d(96, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)

--------------------------------
          Pruning Plan
--------------------------------
User pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)) => ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)), Index=[1, 3, 5, 12, 13, 14, 23, 25, 26, 31, 32, 33], metric={'#params': 1152}]

Coupled pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 36, kernel_size=(1, 1), stride=(1, 1), bias=False)) => BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(36, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)), Index=[1, 3, 5, 12, 13, 14, 23, 25, 26, 31, 32, 33], metric={'#params': 24}]
[ [DEP] BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(36, eps=0.001, momentum=0.03, affine=True, track_

In [17]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [18]:
ckpt_state = {
                "model": model.state_dict(),
            }

torch.save(ckpt_state, 'type1_0.50_all_v2.pth')


